In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime

In [3]:
cities = [
    'Bologna',
    'Firenze',
    'Genova',
    'Milano',
    'Napoli',
    'Palermo',
    'Torino',
    'Roma',
]

list_ = []
for city in cities:
    filename = 'data/generated_files/immobiliare/immobiliare_{}_05_2018.csv'.format(city)
    temp_df = pd.read_csv(filename, index_col=None, header=0)
    list_.append(temp_df)
df = pd.concat(list_)
df = df.reset_index(drop=True)
df.head()

,additionalCharacteristics,area,catasto,climax,condition,constructionYear,energyClass,expensesCondominium,expensesHeating,floorNumber,...,idListing,lat,location,lon,name,placeType,price,propertyType,rooms,url
0,"Terrazzo,Cancello elettrico,Fibra ottica,Video...",82.0,"Classe A/3, rendita € 620",Predisposizione impianto,Ottimo/Ristrutturato,2012.0,A3,159/mese,10/anno,"1° di 3 piani, con ascensore",...,BL427A02B - 02/05/2018,44.4874,NaN,11.3972,"Quadrilocale via due Madonne, 47, Bologna",Appartamento,265.000,Classe immobile media,"4 (2 camere da letto, 2 altro), 1 bagno, cucin...",NaN
1,"Balcone,Cancello elettrico,Videocitofono,Impia...",160.0,"Classe A/3, rendita € 1.800",Assente,Da ristrutturare,1970.0,G,1/mese,1/anno,"3° di 6 piani, con ascensore",...,Rif.V114S - 18/04/2018,44.4965,NaN,11.3381,"Appartamento via San Gervasio, Bologna",Appartamento,430.000,"Intera proprietà, classe immobile signorile","5 (3 camere da letto, 2 altro), 3 bagni, cucin...",NaN
2,"Balcone,Terrazzo,Videocitofono,Porta blindata,...",92.0,NaN,"Autonomo, solo freddo",Nuovo / In costruzione,2017.0,A,60/mese,NaN,"Piano terra di 14 piani, con ascensore",...,A.2.2 (1477622) - 10/05/2018,44.5102,NaN,11.3549,"Trilocale via Donato Creti, Bologna",Appartamento,322.000,"Intera proprietà, classe immobile signorile","3 (2 camere da letto, 1 altro), 2 bagni, cucin...",NaN
3,"Balcone,Terrazzo,Porta blindata",110.0,"Classe A/3, rendita € 258","Autonomo, solo freddo",Nuovo / In costruzione,2017.0,A+,70/mese,NaN,"5° di 14 piani, con ascensore",...,5.1 (1650108) - 10/05/2018,44.5100,NaN,11.3548,"Quadrilocale via Donato Creti, Bologna",Appartamento,387.000,"Intera proprietà, immobile di lusso","4 (2 camere da letto, 2 altro), 2 bagni, cucin...",NaN
4,"Terrazzo,Idromassaggio,Fibra ottica,Videocitof...",150.0,NaN,Autonomo,Ottimo/Ristrutturato,2008.0,NaN,130/mese,NaN,Con ascensore,...,CE141 - 22/01/2018,44.4948,NaN,11.3004,"Attico / Mansarda via Della Certosa adiacenze,...",Attico / Mansarda,720.000,"Intera proprietà, classe immobile signorile","5+ (2 camere da letto, 4 altro), 2 bagni, cuci...",NaN


In [3]:
df.columns

Index(['additionalCharacteristics', 'area', 'catasto', 'climax', 'condition',
       'constructionYear', 'energyClass', 'expensesCondominium',
       'expensesHeating', 'floorNumber', 'garage', 'heating', 'idListing',
       'lat', 'location', 'lon', 'name', 'placeType', 'price', 'propertyType',
       'rooms', 'url'],
      dtype='object')

In [4]:
df = df.dropna(subset=['area'])
df['date_listing'] = pd.to_datetime(df['idListing'].str.split('-').str[-1].str.strip(), format='%d/%m/%Y', errors='ignore')
df = df[df['price'] != 'richiesta']
df = df[df['area'] > 15]
df = df[df['placeType'].isin(('Appartamento', 'Attico / Mansarda', 'Casa indipendente', 'Loft / Open Space', 'Villetta a schiera'))]
df = df[df['condition'] != 'Nuovo / In costruzione']
df.describe()

,area,constructionYear,lat,lon,url
count,81053.000000,57909.000000,79126.000000,79126.000000,0.0
mean,105.555686,1950.041203,43.022831,11.076091,NaN
std,60.639515,65.863681,2.013481,2.166132,NaN
min,16.000000,1.000000,-34.397000,7.606200,NaN
25%,69.000000,1950.000000,41.867800,9.142713,NaN
50%,90.000000,1960.000000,41.969550,12.350950,NaN
75%,125.000000,1970.000000,45.049100,12.524800,NaN
max,960.000000,2019.000000,45.533300,150.644000,NaN


In [5]:
df.groupby(df['date_listing'].dt.year).count()

,additionalCharacteristics,area,catasto,climax,condition,constructionYear,energyClass,expensesCondominium,expensesHeating,floorNumber,...,lat,location,lon,name,placeType,price,propertyType,rooms,url,date_listing
date_listing,,,,,,,,,,,,,,,,,,,,,
2012,6,6,0,6,6,6,6,5,0,6,...,6,0,6,6,6,6,6,6,0,6
2013,14,15,0,13,15,9,15,7,0,14,...,13,2,13,15,15,15,9,15,0,15
2014,52,53,0,35,50,25,49,20,0,52,...,53,0,53,53,53,53,46,52,0,53
2015,141,150,2,72,131,85,137,72,0,138,...,147,3,147,150,150,150,131,145,0,150
2016,751,796,244,462,739,467,682,366,52,747,...,764,32,764,796,796,796,704,782,0,796
2017,8313,8608,4624,6387,8012,6055,6934,5180,1276,8075,...,8260,348,8260,8608,8608,8608,8034,8546,0,8608
2018,68285,71425,42057,56909,65615,51262,58874,47925,12027,67133,...,69883,1542,69883,71425,71425,71425,66212,71227,0,71425


In [6]:
df = df[df['date_listing'] >= datetime.datetime(2017,5,10)]

## Pre-processing

In [7]:
df.loc[:, 'price'] = df['price'].str.replace('.', '')
df['price'] = df['price'].astype(np.float32)

df = df[df['price'] > 20000]

In [8]:
def extract_property_class(x):
    #'Diritto di superficie',
    #'Diritto di superficie, classe immobile economica',
    x = str(x)
    f = x.split(',')
    return f[-1].strip() if 'classe' in f[-1] or len(f) == 2 else np.nan

def extract_property_type(x):
    #'Diritto di superficie',
    #'Diritto di superficie, classe immobile economica',
    x = str(x)
    f = x.split(',')
    return f[0].strip()


df['propertyType'] = df['propertyType'].str.lower()
df['property_class'] = df['propertyType'].apply(lambda x: extract_property_class(x))
df['propertyType'] = df['propertyType'].apply(lambda x: extract_property_type(x))

df.loc[df['property_class'] == 'classe immobile economica', 'property_class'] = 0
df.loc[df['property_class'] == 'classe immobile media', 'property_class'] = 1
df.loc[df['property_class'] == 'classe immobile signorile', 'property_class'] = 2
df.loc[df['property_class'] == 'immobile di lusso', 'property_class'] = 2

df['property_type'] = np.nan
df.loc[df['propertyType'] == 'nuda proprietà', 'property_type'] = 4
df.loc[df['propertyType'] == 'usufrutto', 'property_type'] = 3
df.loc[df['propertyType'] == 'parziale proprietà', 'property_type'] = 2
df.loc[df['propertyType'] == 'diritto di superficie', 'property_type'] = 5
df.loc[df['propertyType'] == 'intera proprietà', 'property_type'] = 1

In [9]:
df = df[df.property_type == 1]

In [10]:
df.loc[:, 'garage'] = df['garage'].apply(lambda x: 1 if 'Box' in str(x) else 0)

In [11]:
def process_land_value(x):
    x = str(x)
    f = x.split(',')
    
    if 'rendita' in f[-1]:
        return f[-1].split(' ')[-1]
    
    return np.nan

df['land_value'] = df['catasto'].str.lower().apply(lambda x: process_land_value(x)).str.replace('.', '').astype(np.float32)

In [12]:
df.loc[:, 'energyClass'] = df['energyClass'].str[:2].str.replace(',', '')
set_energies = {
    'A+':4, 'A':5, 'A1':3, 'A2':2, 'A3':1, 'A4':0, 'B':6, 'C':7, 'D':8, 'E':9, 'F':10, 'G':11, 'No': np.nan
}
for k,v in set_energies.items():
    df.loc[df['energyClass'] == k, 'energyClass'] = v
df['energyClass'] = df['energyClass'].apply(pd.to_numeric, errors='coerce')

In [13]:
df_rooms = df.rooms.str.extract(r"(?:(?P<locali>\d+\+?)(?: (?:local\w(?:\, )?)| \())?(?:(?P<camere>\d+\+?)(?: cam[\w ]+(?:\, )?))?(?:(?P<altro>\d+\+?)(?: altr[\w]+)\)?(?:\, )?)?(?:(?P<bagni>\d+\+?)(?: bagn[\w]+(?:\, )?))?(?:(?:[Cc]ucin)(?P<cucina>[\w ]+))?", expand=True)
df_rooms = pd.get_dummies(df_rooms, columns=['cucina'])
df_rooms.loc[df_rooms.locali.notnull() & df_rooms.locali.str.contains('+', na='', regex=False), 'locali'] = df_rooms.loc[df_rooms.locali.notnull() & df_rooms.locali.str.contains('+', na='', regex=False), 'locali'].str.replace('+', '').astype(np.float32)+1
df_rooms.loc[df_rooms.bagni.notnull() & df_rooms.bagni.str.contains('+', na='', regex=False), 'bagni'] = df_rooms.loc[df_rooms.bagni.notnull() & df_rooms.bagni.str.contains('+', na='', regex=False), 'bagni'].str.replace('+', '').astype(np.float32)+1
df_rooms.loc[df_rooms.altro.notnull() & df_rooms.altro.str.contains('+', na='', regex=False), 'altro'] = df_rooms.loc[df_rooms.altro.notnull() & df_rooms.altro.str.contains('+', na='', regex=False), 'altro'].str.replace('+', '').astype(np.float32)+1
df = pd.concat((df, df_rooms), axis=1)

df['camere'] = df['camere'].astype(np.float32)
df = df[df['camere'] < 100]
df_rooms.head()

,locali,camere,altro,bagni,cucina_a a vista,cucina_a abitabile,cucina_a angolo cottura,cucina_a semi abitabile,cucina_otto
1,5,3,2,3,0,1,0,0,0
4,6,2,4,2,0,1,0,0,0
5,5,4,1,2,0,0,0,1,0
6,6,3,6,2,0,1,0,0,0
7,6,3,3,2,0,1,0,0,0


In [14]:
df = pd.get_dummies(df, columns=['condition'])
df = pd.get_dummies(df, columns=['placeType'])

In [15]:
df['additionalCharacteristics'] = df['additionalCharacteristics'].fillna("")
df['additionalCharacteristics'] = df['additionalCharacteristics'].str.split(',')
df['additionalCharacteristics'] = df['additionalCharacteristics'].map(lambda x: ','.join(sorted(x))).str.split(',')

characteristics = set()

for row in df['additionalCharacteristics'].values:
    characteristics = characteristics.union(set(row))

In [16]:
list_columns = [x for x in characteristics if len(x) > 2]
list_columns

['Taverna',
 'Infissi esterni in vetro / metallo',
 'Portiere intera giornata',
 'Portiere mezza giornata',
 'Infissi esterni in vetro / pvc',
 'Giardino privato',
 'Impianto tv centralizzato',
 'Infissi esterni in doppio vetro / metallo',
 'Parzialmente arredato',
 'Idromassaggio',
 'Impianto tv singolo',
 'Fibra ottica',
 'Infissi esterni in doppio vetro / pvc',
 'Porta blindata',
 'Videocitofono',
 'Impianto tv con parabola satellitare',
 'Cantina',
 'Campo da tennis',
 'Infissi esterni in triplo vetro / pvc',
 'Terrazzo',
 'Piscina',
 'Infissi esterni in triplo vetro / legno',
 'Esposizione doppia',
 'Giardino comune',
 'Infissi esterni in doppio vetro / legno',
 'Infissi esterni in vetro / legno',
 'Impianto allarme',
 'Esposizione esterna',
 'Arredato',
 'Esposizione interna',
 'Mansarda',
 'Camino',
 'Cancello elettrico',
 'Infissi esterni in triplo vetro / metallo',
 'Balcone']

In [17]:
characteristics_table = {
    'Infissi esterni in vetro / legno': 0,
    'Infissi esterni in vetro / pvc': 1,
    'Infissi esterni in vetro / metallo': 2,
    'Infissi esterni in doppio vetro / legno': 3,
    'Infissi esterni in doppio vetro / pvc': 4,
    'Infissi esterni in doppio vetro / metallo': 5,
    'Infissi esterni in triplo vetro / legno': 6,
    'Infissi esterni in triplo vetro / pvc': 7,
    'Infissi esterni in triplo vetro / metallo': 8,
}

portiere_table = {
    'Portiere intera giornata':2,
    'Portiere mezza giornata':1
}

esposizione_table = {
    'Esposizione doppia':3,
    'Esposizione esterna':2,
    'Esposizione interna':1
}

tv_table = {
    'Impianto tv con parabola satellitare':2,
    'Impianto tv singolo': 1,
    'Impianto tv centralizzato':0
}

arredato_table = {
    'Arredato':2,
    'Parzialmente arredato': 1
}

giardino_table = {
    'Giardino comune': 1,
    'Giardino privato': 2
}

cucina_table = {
    'cucina_a angolo cottura': 1,
    'cucina_otto': 1,
    'cucina_a a vista': 2,
    'cucina_a semi abitabile': 3,
    'cucina_a abitabile': 4,
}

condition_table = {
    #'condition_Nuovo / In costruzione': 0,
    'condition_Da ristrutturare': 1,
    'condition_Buono / Abitabile': 2,
    'condition_Ottimo/Ristrutturato': 3
}

In [18]:
df = df.drop('additionalCharacteristics', 1).join(
    pd.get_dummies(
        pd.DataFrame(df.additionalCharacteristics.tolist()).stack()
    ).astype(int).sum(level=0)
)
        
df['infissi'] = np.nan
df['portiere'] = 0
df['giardino'] = 0
df['arredato'] = 0
df['esposizione'] = np.nan
df['tv'] = np.nan
df['kitchen'] = np.nan
df['condition'] = np.nan
df['class'] = np.nan
df['type'] = np.nan
for x in characteristics_table.keys():
    df.loc[df[x] == 1, 'infissi'] = characteristics_table[x]
for x in portiere_table.keys():
    df.loc[df[x] == 1, 'portiere'] = portiere_table[x]
for x in giardino_table.keys():
    df.loc[df[x] == 1, 'giardino'] = giardino_table[x]
for x in arredato_table.keys():
    df.loc[df[x] == 1, 'arredato'] = arredato_table[x]
for x in esposizione_table.keys():
    df.loc[df[x] == 1, 'esposizione'] = esposizione_table[x]
for x in tv_table.keys():
    df.loc[df[x] == 1, 'tv'] = tv_table[x]
for x in condition_table.keys():
    df.loc[df[x] == 1, 'condition'] = condition_table[x]
    
for x in cucina_table.keys():
    df.loc[df[x] == 1, 'kitchen'] = cucina_table[x]

df.head()

,area,catasto,climax,constructionYear,energyClass,expensesCondominium,expensesHeating,floorNumber,garage,heating,...,infissi,portiere,giardino,arredato,esposizione,tv,kitchen,condition,class,type
1,160.0,"Classe A/3, rendita € 1.800",Assente,1970.0,11.0,1/mese,1/anno,"3° di 6 piani, con ascensore",0,"Centralizzato, a radiatori, alimentato a metano",...,NaN,0,0,0,NaN,0.0,4.0,1.0,NaN,NaN
4,150.0,NaN,Autonomo,2008.0,NaN,130/mese,NaN,Con ascensore,0,Centralizzato,...,4.0,0,0,0,2.0,0.0,4.0,3.0,NaN,NaN
5,135.0,NaN,"Autonomo, freddo/caldo",NaN,10.0,70/mese,NaN,3° di 3 piani,0,"Autonomo, a radiatori, alimentato a gas",...,NaN,0,0,0,NaN,NaN,3.0,3.0,NaN,NaN
6,218.0,NaN,Autonomo,1600.0,NaN,125/mese,NaN,1° di 3 piani,1,Autonomo,...,3.0,0,0,2,1.0,0.0,4.0,3.0,NaN,NaN
7,120.0,NaN,Autonomo,1962.0,NaN,250/mese,NaN,"Ultimo di 6 piani, con ascensore",0,Centralizzato,...,NaN,2,0,0,NaN,NaN,4.0,3.0,NaN,NaN


In [19]:
df.loc[df['expensesCondominium'].str.len() <= 1] = np.nan
df.loc[df['expensesCondominium'].str.len() > 1, 'expensesCondominium'] = df[df['expensesCondominium'].str.len() > 1]['expensesCondominium'].str.split('/', expand=True)[0]
df.loc[:, 'expensesCondominium'] = df['expensesCondominium'].str.replace('.', '')
df.head()

,area,catasto,climax,constructionYear,energyClass,expensesCondominium,expensesHeating,floorNumber,garage,heating,...,infissi,portiere,giardino,arredato,esposizione,tv,kitchen,condition,class,type
1,160.0,"Classe A/3, rendita € 1.800",Assente,1970.0,11.0,1,1/anno,"3° di 6 piani, con ascensore",0.0,"Centralizzato, a radiatori, alimentato a metano",...,NaN,0.0,0.0,0.0,NaN,0.0,4.0,1.0,NaN,NaN
4,150.0,NaN,Autonomo,2008.0,NaN,130,NaN,Con ascensore,0.0,Centralizzato,...,4.0,0.0,0.0,0.0,2.0,0.0,4.0,3.0,NaN,NaN
5,135.0,NaN,"Autonomo, freddo/caldo",NaN,10.0,70,NaN,3° di 3 piani,0.0,"Autonomo, a radiatori, alimentato a gas",...,NaN,0.0,0.0,0.0,NaN,NaN,3.0,3.0,NaN,NaN
6,218.0,NaN,Autonomo,1600.0,NaN,125,NaN,1° di 3 piani,1.0,Autonomo,...,3.0,0.0,0.0,2.0,1.0,0.0,4.0,3.0,NaN,NaN
7,120.0,NaN,Autonomo,1962.0,NaN,250,NaN,"Ultimo di 6 piani, con ascensore",0.0,Centralizzato,...,NaN,2.0,0.0,0.0,NaN,NaN,4.0,3.0,NaN,NaN


In [20]:
'''
{'A radiatori',
 'Alimentato a fotovoltaico',
 'Alimentato a metano',
 'Assente', 0
 'Autonomo', 2
 'Centralizzato'} 1
'''

df['newHeating'] = np.nan
df.loc[(~df.heating.isnull()) & (df.heating.str.contains('Assente')), 'newHeating'] = 0
df.loc[(~df.heating.isnull()) & (df.heating.str.contains('Assente')), 'Centralizzato'] = 1
df.loc[(~df.heating.isnull()) & (df.heating.str.contains('Autonomo')), 'newHeating'] = 2
df.loc[(~df.heating.isnull()) & (df.heating.str.contains('Alimentato a metano')), 'newHeating'] = 2
df.loc[(~df.heating.isnull()) & (df.heating.str.contains('Alimentato a fotovoltaico')), 'newHeating'] = 2
df.loc[(~df.heating.isnull()) & (df.heating.str.contains('A radiatori')), 'newHeating'] = 2

In [21]:
# Interrato
# Seminterrato
# Piano rialzato - Ammezzato
# Piano terra
# 1°
# Su più livelli
# altro

# ascensore

df['newFloorNumber'] = np.nan
df.loc[(~df.floorNumber.isnull()), 'newFloorNumber'] = 6
df.loc[(~df.floorNumber.isnull()) & (df.floorNumber.str.contains('Interrato')), 'newFloorNumber'] = 0
df.loc[(~df.floorNumber.isnull()) & (df.floorNumber.str.contains('Seminterrato')), 'newFloorNumber'] = 1
df.loc[(~df.floorNumber.isnull()) & ((df.floorNumber.str.contains('Piano rialzato')) | (df.floorNumber.str.contains('Ammezzato'))), 'newFloorNumber'] = 2
df.loc[(~df.floorNumber.isnull()) & (df.floorNumber.str.contains('Piano terra')), 'newFloorNumber'] = 3
df.loc[(~df.floorNumber.isnull()) & (df.floorNumber.str.contains('1°')), 'newFloorNumber'] = 4
df.loc[(~df.floorNumber.isnull()) & (df.floorNumber.str.contains('livelli')), 'newFloorNumber'] = 5

# ascensore
df['ascensore'] = np.nan
df.loc[(~df.floorNumber.isnull()) & (df.floorNumber.str.contains('con ascensore')), 'ascensore'] = 1

In [22]:
df = df.rename(columns={'area': 'sqmt'})
df['has_terrace'] = df[['Balcone', 'Terrazzo']].head(10).sum(1)
    
# Default values
df.loc[df['Mansarda'].isnull(), 'Mansarda'] = 0
df.loc[df['Idromassaggio'].isnull(), 'Idromassaggio'] = 0
df.loc[df['Camino'].isnull(), 'Camino'] = 0
df.loc[df['locali'].isnull(), 'locali'] = df.loc[df['locali'].isnull(), ['altro', 'camere', 'bagni']].fillna(0).sum(1) + (df.loc[df['locali'].isnull(), 'kitchen'].fillna(0) >= 3).astype(int)

In [23]:
columns = ['sqmt', 
           'constructionYear', 'energyClass', 'expensesCondominium', 'price',
           'newFloorNumber', 'newHeating', 'portiere', 'infissi', 'giardino', 'tv', 'arredato', 'esposizione', 'kitchen',
           'condition', 'land_value',
           
       'Piscina', 
       'Idromassaggio',
       'Mansarda',
       'Cantina',
           'Camino',
       'has_terrace', 'garage', 
          
           'placeType_Appartamento', 'placeType_Attico / Mansarda',
       'placeType_Casa indipendente', 'placeType_Loft / Open Space',
       'placeType_Villetta a schiera', 
           'property_class', 'property_type',
           
           'locali', 'camere', 'altro', 'bagni', 
           
          ]
df[columns] = df[columns].astype(np.float32)
df[columns].head()

,sqmt,constructionYear,energyClass,expensesCondominium,price,newFloorNumber,newHeating,portiere,infissi,giardino,...,placeType_Attico / Mansarda,placeType_Casa indipendente,placeType_Loft / Open Space,placeType_Villetta a schiera,property_class,property_type,locali,camere,altro,bagni
1,160.0,1970.0,11.0,1.0,430000.0,6.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,2.0,1.0,5.0,3.0,2.0,3.0
4,150.0,2008.0,NaN,130.0,720000.0,6.0,NaN,0.0,4.0,0.0,...,1.0,0.0,0.0,0.0,2.0,1.0,6.0,2.0,4.0,2.0
5,135.0,NaN,10.0,70.0,305000.0,6.0,2.0,0.0,NaN,0.0,...,1.0,0.0,0.0,0.0,NaN,1.0,5.0,4.0,1.0,2.0
6,218.0,1600.0,NaN,125.0,870000.0,4.0,2.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,2.0,1.0,6.0,3.0,6.0,2.0
7,120.0,1962.0,NaN,250.0,445000.0,6.0,NaN,2.0,NaN,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,6.0,3.0,3.0,2.0


In [24]:
df[columns].describe()

,sqmt,constructionYear,energyClass,expensesCondominium,price,newFloorNumber,newHeating,portiere,infissi,giardino,...,placeType_Attico / Mansarda,placeType_Casa indipendente,placeType_Loft / Open Space,placeType_Villetta a schiera,property_class,property_type,locali,camere,altro,bagni
count,60623.000000,48003.000000,51079.000000,4.263100e+04,6.062300e+04,58286.000000,32463.000000,60623.000000,27840.000000,60623.000000,...,60623.000000,60623.000000,60623.000000,60623.000000,56845.000000,60623.0,60623.000000,60623.000000,56696.00000,60147.000000
mean,105.850037,1949.983398,10.369154,1.507150e+04,3.742299e+05,4.958206,1.811539,0.271992,2.567134,0.260792,...,0.045115,0.011316,0.008743,0.011860,1.296666,1.0,3.519638,2.165696,1.57355,1.514822
std,58.931396,65.355530,1.301350,2.079448e+06,3.902872e+06,1.418648,0.584307,0.671953,1.939590,0.565916,...,0.207558,0.105773,0.093093,0.108258,0.624977,0.0,1.328797,1.176952,8.32677,0.714737
min,16.000000,1000.000000,0.000000,0.000000e+00,2.090000e+04,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.000000,1.000000,1.00000,1.000000
25%,70.000000,1950.000000,10.000000,5.000000e+01,1.450000e+05,4.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,3.000000,1.000000,1.00000,1.000000
50%,90.000000,1960.000000,11.000000,8.000000e+01,2.400000e+05,6.000000,2.000000,0.000000,3.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,3.000000,2.000000,1.00000,1.000000
75%,127.000000,1970.000000,11.000000,1.320000e+02,3.980000e+05,6.000000,2.000000,0.000000,4.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,2.000000,1.0,4.000000,3.000000,2.00000,2.000000
max,960.000000,2019.000000,11.000000,3.237030e+08,6.750000e+08,6.000000,2.000000,2.000000,8.000000,2.000000,...,1.000000,1.000000,1.000000,1.000000,2.000000,1.0,32.000000,84.000000,1960.00000,4.000000


## Attach sez2011

In [25]:
gdf_census = gpd.read_file("data/generated_files/dsaa_census_areas.shp")
gdf_census.head()

,sez2011,geometry
0,0012720001226,"POLYGON ((7.660425319648821 45.05038578651656,..."
1,0012720000091,"POLYGON ((7.683635179963704 45.06923328757473,..."
2,0012720001837,"POLYGON ((7.679657280480781 45.10092137770937,..."
3,0012720000309,"POLYGON ((7.670415391208696 45.06782455187603,..."
4,0012720000398,"POLYGON ((7.671571572791478 45.05805060715785,..."


In [26]:
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(df.lon, df.lat)]
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(df.drop(['lon', 'lat'], axis=1), crs=crs, geometry=geometry)
gdf.head()

,sqmt,catasto,climax,constructionYear,energyClass,expensesCondominium,expensesHeating,floorNumber,garage,heating,...,kitchen,condition,class,type,newHeating,Centralizzato,newFloorNumber,ascensore,has_terrace,geometry
1,160.0,"Classe A/3, rendita € 1.800",Assente,1970.0,11.0,1.0,1/anno,"3° di 6 piani, con ascensore",0.0,"Centralizzato, a radiatori, alimentato a metano",...,4.0,1.0,NaN,NaN,NaN,NaN,6.0,1.0,1.0,POINT (11.3381 44.4965)
4,150.0,NaN,Autonomo,2008.0,NaN,130.0,NaN,Con ascensore,0.0,Centralizzato,...,4.0,3.0,NaN,NaN,NaN,NaN,6.0,NaN,1.0,POINT (11.3004 44.4948)
5,135.0,NaN,"Autonomo, freddo/caldo",NaN,10.0,70.0,NaN,3° di 3 piani,0.0,"Autonomo, a radiatori, alimentato a gas",...,3.0,3.0,NaN,NaN,2.0,NaN,6.0,NaN,1.0,POINT (11.38 44.47600000000001)
6,218.0,NaN,Autonomo,1600.0,NaN,125.0,NaN,1° di 3 piani,1.0,Autonomo,...,4.0,3.0,NaN,NaN,2.0,NaN,4.0,NaN,1.0,POINT (11.3501 44.4901)
7,120.0,NaN,Autonomo,1962.0,NaN,250.0,NaN,"Ultimo di 6 piani, con ascensore",0.0,Centralizzato,...,4.0,3.0,NaN,NaN,NaN,NaN,6.0,1.0,1.0,POINT (11.3304 44.498)


In [27]:
df_joined = gpd.sjoin(gdf, gdf_census, how="inner", op='within')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/function_base.py:2831: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [28]:
df_joined.columns

Index(['sqmt', 'catasto', 'climax', 'constructionYear', 'energyClass',
       'expensesCondominium', 'expensesHeating', 'floorNumber', 'garage',
       'heating', 'idListing', 'location', 'name', 'price', 'propertyType',
       'rooms', 'url', 'date_listing', 'property_class', 'property_type',
       'land_value', 'locali', 'camere', 'altro', 'bagni', 'cucina_a a vista',
       'cucina_a abitabile', 'cucina_a angolo cottura',
       'cucina_a semi abitabile', 'cucina_otto', 'condition_Buono / Abitabile',
       'condition_Da ristrutturare', 'condition_Ottimo/Ristrutturato',
       'placeType_Appartamento', 'placeType_Attico / Mansarda',
       'placeType_Casa indipendente', 'placeType_Loft / Open Space',
       'placeType_Villetta a schiera', '', 'Arredato', 'Balcone', 'Camino',
       'Campo da tennis', 'Cancello elettrico', 'Cantina',
       'Esposizione doppia', 'Esposizione esterna', 'Esposizione interna',
       'Fibra ottica', 'Giardino comune', 'Giardino privato', 'Idromassaggio

## Attach OMI context

In [1]:
'''from os import listdir
from os.path import isfile, join

omi_path = 'data/omi'
omi_files = [f for f in listdir(omi_path) if isfile(join(omi_path, f)) and '.csv' in f]

list_ = []
for f in omi_files:
    filename = omi_path+'/'+f
    temp_df = pd.read_csv(filename, index_col=None, header=1, delimiter=';')
    list_.append(temp_df)
omi_df = pd.concat(list_)
omi_df = omi_df.reset_index(drop=True)

omi_df['istat'] = omi_df['Comune_ISTAT'].astype(str).str[-5:]
omi_df = omi_df[(omi_df['Stato'] == 'NORMALE') & (omi_df['Descr_Tipologia'] == 'Abitazioni civili')]
omi_df['med_range'] = omi_df['Compr_max'] - omi_df['Compr_min']

omi_df = omi_df[['istat', 'Fascia', 'med_range']].groupby(['istat', 'Fascia'], as_index=False).mean()

omi_df = omi_df.pivot(index='istat', columns='Fascia', values='med_range')

omi_df = omi_df[['B', 'C', 'D', 'E']]
omi_df.columns = ['omi_{}'.format(x) for x in omi_df.columns]

omi_df.head()

df_joined['istat'] = df_joined.sez2011.str[1:6]
df_joined = pd.merge(df_joined, omi_df, left_on='istat', right_index=True, how='left')
'''

"from os import listdir\nfrom os.path import isfile, join\n\nomi_path = 'data/omi'\nomi_files = [f for f in listdir(omi_path) if isfile(join(omi_path, f)) and '.csv' in f]\n\nlist_ = []\nfor f in omi_files:\n    filename = omi_path+'/'+f\n    temp_df = pd.read_csv(filename, index_col=None, header=1, delimiter=';')\n    list_.append(temp_df)\nomi_df = pd.concat(list_)\nomi_df = omi_df.reset_index(drop=True)\n\nomi_df['istat'] = omi_df['Comune_ISTAT'].astype(str).str[-5:]\nomi_df = omi_df[(omi_df['Stato'] == 'NORMALE') & (omi_df['Descr_Tipologia'] == 'Abitazioni civili')]\nomi_df['med_range'] = omi_df['Compr_max'] - omi_df['Compr_min']\n\nomi_df = omi_df[['istat', 'Fascia', 'med_range']].groupby(['istat', 'Fascia'], as_index=False).mean()\n\nomi_df = omi_df.pivot(index='istat', columns='Fascia', values='med_range')\n\nomi_df = omi_df[['B', 'C', 'D', 'E']]\nomi_df.columns = ['omi_{}'.format(x) for x in omi_df.columns]\n\nomi_df.head()\n\ndf_joined['istat'] = df_joined.sez2011.str[1:6]\ndf

## Save

In [32]:
df_joined = df_joined.set_index('sez2011')
df_joined[columns + omi_df.columns.tolist()].to_parquet('data/generated_files/selling_houses.parquet')